In [1]:
#没有进行数字图像处理操作前dlib包成功识别到人脸的概率
#目前对example_4的识别效果不好，成功识别率只有76.67%
import cv2
import dlib

# 加载前置人脸检测器
detector = dlib.get_frontal_face_detector()

# 打开视频文件
video = cv2.VideoCapture('./data/example_4.avi')

# 初始化帧数计数器和识别到的人脸计数器
frame_count = 0
faces_detected = 0

while True:
    # 逐帧读取视频
    ret, frame = video.read()

    # 如果读取成功
    if ret:
        # 转换当前帧为灰度图像
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #对图像进行处理的操作放在这边
        # 使用检测器检测人脸
        faces = detector(gray)
        
        # 记录是否检测到人脸
        if len(faces) > 0:
            faces_detected += 1

        # 绘制人脸边框
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            
            # 在原始帧上画矩形以标记人脸
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)      
        # 显示带有标记人脸的帧
        cv2.imshow('Frame with Detected Faces', frame)

        # 如果按下'q'键，就退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  # 如果没有下一帧了，退出循环
    frame_count += 1

# 释放视频文件和销毁所有窗口
video.release()
cv2.destroyAllWindows()

# 打印视频中检测到人脸的帧数占总帧数的比例
print(f'Detected faces in {faces_detected} out of {frame_count} frames.')
print(f'{(faces_detected / frame_count) * 100:.2f}% of all frames have faces detected.')

Detected faces in 46 out of 60 frames.
76.67% of all frames have faces detected.


In [13]:
import cv2
import dlib
import numpy as np

# 加载前置人脸检测器
detector = dlib.get_frontal_face_detector()

# 打开视频文件
video = cv2.VideoCapture('./data/example_4.avi')

# 初始化帧数计数器和识别到的人脸计数器
frame_count = 0
faces_detected = 0

while True:
    # 逐帧读取视频
    ret, frame = video.read()

    # 如果读取成功
    if ret:
        # 转换当前帧为灰度图像
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 自适应直方图均衡化
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        clahe_gray = clahe.apply(gray)

        # 对CLAHE结果应用高斯模糊，以减少噪声
        blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整88.83

        gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

        # 然后应用拉普拉斯算子进行边缘检测
        ddepth = cv2.CV_16S
        kernel_size = 3
        laplacian = cv2.Laplacian(gaussian_blur, ddepth, ksize=kernel_size)
        laplacian_abs = cv2.convertScaleAbs(laplacian)

        # 将增强的边缘与原CLAHE图像叠加
        alpha = 0.7
        beta = 1 - alpha
        enhanced_edges = cv2.addWeighted(clahe_gray, alpha, laplacian_abs, beta, 0)


        # 使用检测器检测人脸
        faces = detector(enhanced_edges)
        
        # 记录是否检测到人脸
        if len(faces) > 0:
            faces_detected += 1

        # 绘制人脸边框
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            
            # 在原始帧上画矩形以标记人脸
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
        
        # 显示带有标记人脸的帧
        cv2.imshow('Frame with Detected Faces', frame)

        # 如果按下'q'键，就退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  # 如果没有下一帧了，退出循环

    frame_count += 1

# 释放视频文件和销毁所有窗口
video.release()
cv2.destroyAllWindows()

# 打印视频中检测到人脸的帧数占总帧数的比例
print(f'Detected faces in {faces_detected} out of {frame_count} frames.')
print(f'{(faces_detected / frame_count) * 100:.2f}% of all frames have faces detected.')

Detected faces in 52 out of 60 frames.
86.67% of all frames have faces detected.


In [15]:

import cv2
import dlib
import numpy as np

# 加载前置人脸检测器
detector = dlib.get_frontal_face_detector()

# 打开视频文件
video = cv2.VideoCapture('./data/example_4.avi')

# 初始化帧数计数器和识别到的人脸计数器
frame_count = 0
faces_detected = 0

while True:
    # 逐帧读取视频
    ret, frame = video.read()

    # 如果读取成功
    if ret:
        # 转换当前帧为灰度图像
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 自适应直方图均衡化
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        clahe_gray = clahe.apply(gray)

        # 对CLAHE结果应用高斯模糊，以减少噪声
        blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整

        gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

   
        # 使用Canny算法进行边缘检测
        low_threshold = 50  # Canny边缘检测的低阈值
        high_threshold = 150  # Canny边缘检测的高阈值
        canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

        # 将Canny边缘图像转换为原图像空间的大小以便叠加
        canny_edges_colored = cv2.cvtColor(canny_edges, cv2.COLOR_GRAY2BGR)

        # 将增强的边缘与原CLAHE图像叠加
        alpha = 0.8
        beta = 1 - alpha
        enhanced_edges = cv2.addWeighted(clahe_gray, alpha, canny_edges, beta, 0)
        

        # 使用检测器检测人脸
        faces = detector(enhanced_edges)
        
        # 记录是否检测到人脸
        if len(faces) > 0:
            faces_detected += 1

        # 绘制人脸边框
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            
            # 在原始帧上画矩形以标记人脸
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
        
        # 显示带有标记人脸的帧
        cv2.imshow('Frame with Detected Faces', frame)

        # 如果按下'q'键，就退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  # 如果没有下一帧了，退出循环

    frame_count += 1

# 释放视频文件和销毁所有窗口
video.release()
cv2.destroyAllWindows()

# 打印视频中检测到人脸的帧数占总帧数的比例
print(f'Detected faces in {faces_detected} out of {frame_count} frames.')
print(f'{(faces_detected / frame_count) * 100:.2f}% of all frames have faces detected.')

Detected faces in 56 out of 60 frames.
93.33% of all frames have faces detected.


In [2]:
#在canny的基础加上形态学操作，成功识别率达到96.67%
import cv2
import dlib
import numpy as np

# 加载前置人脸检测器
detector = dlib.get_frontal_face_detector()

# 打开视频文件
video = cv2.VideoCapture('./data/example_4.avi')

# 初始化帧数计数器和识别到的人脸计数器
frame_count = 0
faces_detected = 0

while True:
    # 逐帧读取视频
    ret, frame = video.read()

    # 如果读取成功
    if ret:
        # 转换当前帧为灰度图像
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # 自适应直方图均衡化
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        clahe_gray = clahe.apply(gray)

        # 对CLAHE结果应用高斯模糊，以减少噪声
        blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整
        gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

        # 使用Canny算法进行边缘检测
        low_threshold = 50  # Canny边缘检测的低阈值
        high_threshold = 150  # Canny边缘检测的高阈值
        canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

        # 进行形态学变换 - 闭操作
        kernel = np.ones((5, 5), np.uint8)
        closing = cv2.morphologyEx(canny_edges, cv2.MORPH_CLOSE, kernel)

        # 将闭操作后的边缘与原CLAHE图像进行叠加
        # 转换闭操作后的边缘图像的数据类型，使其可以与原图像进行加权叠加
        closing_for_combination = cv2.convertScaleAbs(closing)
        enhanced_edges = cv2.addWeighted(clahe_gray, 0.7, closing_for_combination, 0.3, 0)
        

        # 使用检测器检测人脸
        faces = detector(enhanced_edges)
        
        # 记录是否检测到人脸
        if len(faces) > 0:
            faces_detected += 1

        # 绘制人脸边框
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            
            # 在原始帧上画矩形以标记人脸
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
        
        # 显示带有标记人脸的帧
        cv2.imshow('Frame with Detected Faces', frame)

        # 如果按下'q'键，就退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  # 如果没有下一帧了，退出循环

    frame_count += 1

# 释放视频文件和销毁所有窗口
video.release()
cv2.destroyAllWindows()

# 打印视频中检测到人脸的帧数占总帧数的比例
print(f'Detected faces in {faces_detected} out of {frame_count} frames.')
print(f'{(faces_detected / frame_count) * 100:.2f}% of all frames have faces detected.')

Detected faces in 58 out of 60 frames.
96.67% of all frames have faces detected.
